# Overview

This notebook shows how to train and test a neural network on the BELLA data.

In order to run this notebook, you need to produce first produce CSV files for the training and testing data. (See the folder `experimental data`.)

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import transformer
import torch
from Neural_Net_Classes import NN as NN

<h2>Loading Split Data</h2>

In [14]:
# Load the CSV file
training_set_df = pd.read_csv('training_set_1.csv')

# Access the arrays
z_training_set = training_set_df['z_target (m)'].values
TOD_training_set = training_set_df['TOD (s^3)'].values
protons_training_set = training_set_df['n_protons (1/sr)'].values

In [15]:
# Load the CSV file
test_set_df = pd.read_csv('test_set_1.csv')

# Access the arrays
z_test_set = test_set_df['z_target (m)'].values
TOD_test_set = test_set_df['TOD (s^3)'].values
protons_test_set = test_set_df['n_protons (1/sr)'].values

<h2>Visualizing Split Datasets</h2>

In [ ]:
plt.clf()
ax = plt.figure().add_subplot(projection='3d')

ax.scatter( TOD_training_set, z_training_set,protons_training_set, c='r',alpha=0.3, label='Training Set')
ax.scatter( TOD_test_set, z_test_set,protons_test_set, c='b', alpha=0.3, label='Testing Set')
ax.view_init(elev=40., azim=40, roll=0)
plt.xlabel('TOD')
plt.ylabel('z_target')
plt.legend()

<h2>Normalizing Data</h2>

In [17]:
#Setting Bounds
z_bounds = torch.tensor([-150, 150])
TOD_bounds = torch.tensor([-80e3, 80e3])
protons_bounds = torch.tensor([min(protons_training_set), max(protons_training_set)])

#define transformers
transformer_z = transformer.Transformer(z_bounds.reshape(2,1), transform_type = 'normalize')
transformer_TOD = transformer.Transformer(TOD_bounds.reshape(2,1), transform_type = 'normalize')
transformer_protons = transformer.Transformer(protons_bounds.reshape(2,1), transform_type = 'normalize')

#Full normalization process
def normalization(array, transformer):
    array = np.array(array).reshape(-1,1)
    array = torch.tensor(array)
    norm = transformer.forward(array)
    return norm

In [18]:
#Normalize datasets 1
norm_z_train_set = normalization(z_training_set, transformer_z)
norm_TOD_train_set = normalization(TOD_training_set, transformer_TOD)
norm_protons_train_set = normalization(protons_training_set, transformer_protons)
norm_z_test_set = normalization(z_test_set, transformer_z)
norm_TOD_test_set = normalization(TOD_test_set, transformer_TOD)
norm_protons_test_set = normalization(protons_test_set, transformer_protons)

<h2>Visualizing Normalized Data</h2>

In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for training set
ax.scatter(norm_TOD_train_set, norm_z_train_set, norm_protons_train_set, label='Training Set', alpha=0.7)
# Scatter plot for testing set
ax.scatter(norm_TOD_test_set, norm_z_test_set, norm_protons_test_set, label='Test Set', alpha=0.7)
ax.view_init(elev=40., azim=40, roll=0)
# Set labels and title
ax.set_xlabel('Normalized TOD')
ax.set_ylabel('Normalized Z')
ax.set_zlabel('Normalized Protons')

# Add legend
ax.legend()
# Show plot
plt.show()

<h1>Neural Network Framework</h1>

<h2>Build and Train Neural Networks</h2>

In [ ]:
net = NN()
net.train_model(norm_z_train_set, norm_TOD_train_set, norm_protons_train_set)
net.plot_loss()
net.test_model(norm_z_test_set, norm_TOD_test_set, norm_protons_test_set)

In [21]:
train_predictions = net.predict(norm_z_train_set, norm_TOD_train_set)
test_predictions = net.predict(norm_z_test_set, norm_TOD_test_set)

<h2>Plotting Predictions</h2>

In [ ]:
fig, ax = plt.subplots()

ax.scatter(norm_z_train_set, norm_protons_train_set, label='Training Set 1')
ax.scatter(norm_z_test_set, norm_protons_test_set, label='Test Set 1')

ax.scatter(train_predictions['Z_target'], train_predictions['predictions'], label='predictions', s=50, facecolors='none', edgecolors='r')
ax.scatter(test_predictions['Z_target'], test_predictions['predictions'], s=50, facecolors='none', edgecolors='r')

plt.title("n_protons predictions")
plt.xlabel('z_target (m)')
plt.ylabel('Number of protons (1/sr)')
plt.legend()

In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for training set
ax.scatter(norm_TOD_train_set, norm_z_train_set, norm_protons_train_set, label='Training Set 1', alpha=0.7)
ax.scatter(norm_TOD_test_set, norm_z_test_set, norm_protons_test_set, label='Test Set 1', alpha=0.7)

ax.scatter(train_predictions['TOD'], train_predictions['Z_target'], train_predictions['predictions'], label='predictions 1', s=50, facecolors='none', edgecolors='r')
ax.scatter(test_predictions['TOD'], test_predictions['Z_target'], test_predictions['predictions'], s=50, facecolors='none', edgecolors='r')

ax.view_init(elev=40., azim=40, roll=0)
# Set labels and title
ax.set_title('Experimental Data v Predictions')
ax.set_xlabel('TOD')
ax.set_ylabel('z_target')
ax.set_zlabel('n Protons')

# Add legend
ax.legend()
# Show plot
plt.show()